<a href="https://colab.research.google.com/github/bernalisi/healthcare_semantic_search_algorithm/blob/main/BioBERT_semantich_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing packages
!pip install qdrant-client
!pip install transformers
!pip install spacy
!pip install https://huggingface.co/spacy/en_core_web_sm/resolve/main/en_core_web_sm-any-py3-none-any.whl
!pip install torch
!pip install datasets
!pip install torch torchvision torchaudio

# Loading libraries
from datasets import load_dataset
import spacy
import torch
import re
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from qdrant_client import QdrantClient, models
import numpy as np
from qdrant_client import models
from tqdm.auto import tqdm
from qdrant_client import QdrantClient
import json

In [ ]:
# Link google drive to google colab
from google.colab import drive
drive.mount('/content/drive')

# loading bar
tqdm.pandas(desc="Progress:")

Mounted at /content/drive


In [ ]:
# Load models
nlp = spacy.load("en_core_web_sm")  # Load spaCy English model (download if needed)
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
model = AutoModel.from_pretrained("dmis-lab/biobert-base-cased-v1.1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
# Save dataset (use this code when there is a need to save the csv)

# # Set filename and path (replace with your desired values)
# filename = 'pre-processed data for bioBERT.csv'
# path = 'write path here'  # Replace with your preferred location

# # Export to CSV (without index by default)
# df.to_csv(path + filename)

### **Functions to process the dataset**

**Pre-processing dataset**

In [ ]:
def preprocess_text(text):
    """Preprocesses text using spaCy, handling both strings and floats.

    Args:
        text (str or float): The text to be processed.

    Returns:
        str or float: The processed text (or the original float if not a string).
    """

    if not isinstance(text, str):  # Check if it's a string
        return text  # Return the float value as it is (no processing)
    else:
        # Normalization (lowercase and remove unnecessary punctuation)
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)

        # spaCy processing
        doc = nlp(text)
        processed_tokens = []
        for token in doc:
            if token.is_stop or token.is_punct:
                continue
            elif token.ent_type_:  # Check if the token is a named entity
                processed_tokens.append(token.text)  # Preserve named entities as they are
            else:
                processed_tokens.append(token.lemma_)  # Lemmatize non-entity tokens

        # Re-join tokens
        return ' '.join(processed_tokens)

# Assuming df is your DataFrame

# df['Processed_Title'] = df['Title'].progress_apply(preprocess_text)
# df['Processed_Description'] = df['Study description'].progress_apply(preprocess_text)
# df['Processed_Country'] = df['Study countries'].progress_apply(preprocess_text)


Progress::   0%|          | 0/2791 [00:00<?, ?it/s]

Progress::   0%|          | 0/2791 [00:00<?, ?it/s]

Progress::   0%|          | 0/2791 [00:00<?, ?it/s]

**Vectorize data**

In [ ]:
def vectorize_text(text, tokenizer, model):
    """Vectorizes text data using a tokenizer and model, handling potential floats.

    Args:
        text (str or float): The data to be vectorized.
        tokenizer: The tokenizer to use for text tokenization.
        model: The model to use for generating vector representations.

    Returns:
        torch.Tensor or float: The vector representation of the text (or the original float value).
    """

    if not isinstance(text, str):
        # If not a string, return the float value as a tensor (assuming it's a number)
        return torch.tensor(float(text)).to(model.device)  # Convert to tensor and move to device

    # Text processing (unchanged)
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    # Use the mean of the last hidden states as the vector representation
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu()

# Applying the function (unchanged)
# df['Vectorized_Study_Description'] = df['Processed_Description'].progress_apply(lambda x: vectorize_text(x, tokenizer, model))
# df['Vectorized_Title'] = df['Processed_Title'].progress_apply(lambda x: vectorize_text(x, tokenizer, model))
# df['Vectorized_Study_Countries'] = df['Processed_Country'].progress_apply(lambda x: vectorize_text(x, tokenizer, model))


Progress::   0%|          | 0/2791 [00:00<?, ?it/s]

Progress::   0%|          | 0/2791 [00:00<?, ?it/s]

Progress::   0%|          | 0/2791 [00:00<?, ?it/s]

##**Execution begins here**

In [ ]:
# The dataset URL
dataset_url = "write path here"

# Load the dataset
df = pd.read_csv(dataset_url)

API Key x cluster EMA_Catalogue_Convex = oMxoq3F4CG4BNhebC7Fu5fILrQ1icsFhfj101BGPmX9Z__-4Dw7x_g

In [ ]:
qdrant_client = QdrantClient(
    url="", # Enter qdrant url
    api_key="",  # Enter api key
)

**Da testare ancora**

In [ ]:
# Define the collection name and vector size (depends on BioBERT model, e.g., 768 for base models)
collection_name = "semantic_search_EMA_Catalogue_Convex"
vector_size = 768  # Adjust based on your BioBERT model

# Create a collection
qdrant_client.recreate_collection(
    collection_name=collection_name,
    vectors_config = models.VectorParams(
        size=vector_size,  # Vector size is defined by used model
        distance=models.Distance.COSINE,
    )
)

True

**Create a vectorized data list**

In [ ]:
vectorized_data = [
    {
        'id': row['Study ID'],
        'vector': np.concatenate([
            np.fromstring(row['Vectorized_Title'].replace('\n', '').replace('[', '').replace(']', ''), sep=' '),
            np.fromstring(row['Vectorized_Study_Description'].replace('\n', '').replace('[', '').replace(']', ''), sep=' '),
            np.fromstring(row['Vectorized_Study_Countries'].replace('\n', '').replace('[', '').replace(']', ''), sep=' ')
        ]),
        'title': row['Title'],
        'study_description': row['Study description'] if pd.notnull(row['Study description']) else "No description",  # Handling NaN here
        'study_countries': row['Study countries']
    }
    for _, row in df.iterrows()
]

<ipython-input-109-f90b1b9d91c8>:5: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  np.fromstring(row['Vectorized_Title'].replace('\n', '').replace('[', '').replace(']', ''), sep=' '),
<ipython-input-109-f90b1b9d91c8>:6: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  np.fromstring(row['Vectorized_Study_Description'].replace('\n', '').replace('[', '').replace(']', ''), sep=' '),
<ipython-input-109-f90b1b9d91c8>:7: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  np.fromstring(row['Vectorized_Study_Countries'].replace('\n', '').replace('[', '').replace(']', ''), sep=' ')


In [ ]:
# Recursive function to convert ndarray objects to lists in a nested structure
def convert_ndarray(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()  # Convert ndarray to list
    elif isinstance(obj, dict):
        return {k: convert_ndarray(v) for k, v in obj.items()}  # Apply recursively to dictionaries
    elif isinstance(obj, list):
        return [convert_ndarray(v) for v in obj]  # Apply recursively to lists
    else:
        return obj

vectorized_data_converted = convert_ndarray(vectorized_data)

# Assuming vectorized_data_converted is ready
vectorized_data_json = json.dumps(vectorized_data_converted, indent=4)

# Save to a file
with open('vectorized_data.json', 'w') as f:
    f.write(vectorized_data_json)

print("JSON data has been saved to 'vectorized_data.json'.")


JSON data has been saved to 'vectorized_data.json'.


In [ ]:
filename = 'json_file.json'

# Use 'with' statement to open a file and ensure proper closure after writing
with open(filename, 'w') as file:
    # Use json.dump() to write the JSON data to the file
    # Optional: use 'indent' parameter for pretty-printing, making the file human-readable
    json.dump(vectorized_data_json, file, indent=4)

print(f"JSON data has been saved to '{filename}'.")

JSON data has been saved to 'my_large_json_file.json'.


In [ ]:
print(snippet)

remic Syndrome (aHUS) who are treated with Soliris\u00ac\u00c6 (Eculizumab)",
  


In [ ]:
len(vectorized_data)

2791

In [ ]:
for data_dict in vectorized_data:
  for key, value in data_dict.items():
    print(key, value)

Output streaming troncato alle ultime 5000 righe.
id 48413
vector []
title Patterns of anti-CGRP mAbs use and triptan consumption before and after anti-CGRP treatment initiation: a descriptive drug utilization study in Tuscany region, Italy
study_description In 2019, three monoclonal antibodies(mAbs) that target calcitonin gene-related peptide (CGRP),or its receptor,were approved in Italy for both episodic and chronic migraine prophylaxis:erenumab,galcanezumab and fremanezumab. Given the recent introduction of anti-CGRP mAbs on the market, real-world evidence on utilization patterns and the characteristics of patients using anti-CGRP mAbs is still scarce. Therefore, this study aims to re-use routinely collected healthcare data from the administrative database of Tuscany region, Italy, in order to: 1) provide real-world evidence on patients initiating an anti-CGRP mAb, 2) describe the utilization pattern of these medications up to 15 months of follow-up and 3) observe the consumption of

**Upload vectors to the cloud**

In [ ]:
collection_name = "semantic_search_EMA_Catalogue_Convex"

def upload_batch(batch):
    # Upload the batch to Qdrant
     qdrant_client.upload_points(
        collection_name=collection_name,
        points=[
           models.PointStruct(
                id=int(data_point["id"]),
                vector=data_point["vector"].tolist(),  # Convert numpy array to list
                payload={
                    "title": data_point["title"],
                    "study_description": data_point["study_description"],
                    "study_countries": data_point["study_countries"]
                }
            )
            for data_point in batch
        ],
    )


# # # Batch size
batch_size = 50

# # # Calculate 60% of the dataset size
subset_size = int(len(vectorized_data) * 1)

# # # Upload the dataset
for i in range(0, subset_size, batch_size):
    batch = vectorized_data[i:i + batch_size]
    upload_batch(batch)
    print(f"Total no of batches {subset_size/batch_size}")
    print(f"Uploaded batch {i // batch_size}")

Total no of batches 55.82
Uploaded batch 0
Total no of batches 55.82
Uploaded batch 1
Total no of batches 55.82
Uploaded batch 2
Total no of batches 55.82
Uploaded batch 3
Total no of batches 55.82
Uploaded batch 4
Total no of batches 55.82
Uploaded batch 5
Total no of batches 55.82
Uploaded batch 6
Total no of batches 55.82
Uploaded batch 7
Total no of batches 55.82
Uploaded batch 8
Total no of batches 55.82
Uploaded batch 9
Total no of batches 55.82
Uploaded batch 10
Total no of batches 55.82
Uploaded batch 11
Total no of batches 55.82
Uploaded batch 12
Total no of batches 55.82
Uploaded batch 13
Total no of batches 55.82
Uploaded batch 14
Total no of batches 55.82
Uploaded batch 15
Total no of batches 55.82
Uploaded batch 16
Total no of batches 55.82
Uploaded batch 17
Total no of batches 55.82
Uploaded batch 18
Total no of batches 55.82
Uploaded batch 19
Total no of batches 55.82
Uploaded batch 20
Total no of batches 55.82
Uploaded batch 21
Total no of batches 55.82
Uploaded batch 2

**Search the database**

In [ ]:
def vectorize_query(query, tokenizer, model):
    inputs = tokenizer(query, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

def search_in_qdrant(query, tokenizer, model, top_k=10):
    query_vector = vectorize_query(query, tokenizer, model)

    # Search in Qdrant
    hits = qdrant_client.search(
        collection_name = "semantic_search_EMA_Catalogue_Convex",
        query_vector=query_vector.tolist(),
        limit=top_k,
    )
    return hits

**Result Handling**

In [ ]:
def display_search_results(test_query, tokenizer, model, cutoff_score):
    results = search_in_qdrant(test_query, tokenizer, model)
    print(f"Results returned: {len(results)}")  # Debugging line
    for result in results:
        print(f"Score: {result.score}")  # Debugging line
        if result.score >= cutoff_score:
            print("Title:", result.payload["title"])
            print("Study Description:", result.payload["study description"])
            print("Study Countries:", result.score)
            print("-----------")

In [ ]:
test_query = "Cancer related studies"
display_search_results(test_query, tokenizer, model, 0.90)

Results returned: 0


In [ ]:
vectorized_data = [
    {
        'id': row['Study ID'],
        'vector': np.concatenate([
            np.fromstring(row['Vectorized_Title']),
            np.fromstring(row['Vectorized_Study_Description'].replace('\n', '').replace('[', '').replace(']', ''), sep=' '),
            np.fromstring(row['Vectorized_Study_Countries'].replace('\n', '').replace('[', '').replace(']', ''), sep=' ')
        ]),
        'title': row['Title'],
        'study_description': row['Study description'] if pd.notnull(row['Study description']) else "No description",  # Handling NaN here
        'study_countries': row['Study countries']
    }
    for _, row in df.iterrows()
]

collection_name = "semantic_search_EMA_Catalogue_Convex"

def upload_batch(batch):
    # Upload the batch to Qdrant
     qdrant_client.upload_points(
        collection_name=collection_name,
        points=[
           models.PointStruct(
                id=int(data_point["id"]),
                vector=data_point["vector"].tolist(),  # Convert numpy array to list
                payload={
                    "title": data_point["title"],
                    "study_description": data_point["study_description"],
                    "study_countries": data_point["study_countries"]
                }
            )
            for data_point in batch
        ],
    )

# # Batch size
batch_size = 50

# # Calculate 60% of the dataset size
subset_size = int(len(vectorized_data) * 1)

# # Upload the dataset
for i in range(0, subset_size, batch_size):
    batch = vectorized_data[i:i + batch_size]
    upload_batch(batch)
    print(f"Total no of batches {subset_size/batch_size}")
    print(f"Uploaded batch {i // batch_size}")

def vectorize_query(query, tokenizer, model):
    inputs = tokenizer(query, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

def search_in_qdrant(query, tokenizer, model, top_k=10):
    query_vector = vectorize_query(query, tokenizer, model)

    # Search in Qdrant
    hits = qdrant_client.search(
        collection_name = "semantic_search_EMA_Catalogue_Convex",
        query_vector=query_vector.tolist(),
        limit=top_k,
    )
    return hits

def display_search_results(test_query, tokenizer, model, cutoff_score):
    results = search_in_qdrant(test_query, tokenizer, model)
    for result in results:
        if result.score >= cutoff_score:
            print("Title:", result.payload["title"])
            print("Study Description:", result.payload["study description"])
            print("Study Countries:", result.score)
            print("-----------")

test_query = "Cancer related studies"
display_search_results(test_query, tokenizer, model, 0.90)